<a href="https://colab.research.google.com/github/mifm/examples/blob/master/smt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

In [ ]:
%%capture
!conda create --name hydesign python=3.9.18 smt=2.0

In [4]:
%%capture
%%bash
source activate hydesign

In [6]:
%%capture
!conda install finitediff

In [ ]:
%%capture
!pip install hydesign

In [10]:
import pandas as pd
import numpy as np
from hydesign.assembly.hpp_assembly_constantoutput import hpp_model_constant_output as hpp_model
from hydesign.examples import examples_filepath
import matplotlib.pyplot as plt
from hydesign.Parallel_EGO import get_kwargs, EfficientGlobalOptimizationDriver
import os

In [9]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0)
name = 'France_good_wind'
ex_site = examples_sites.loc[examples_sites.name == name]
longitude = ex_site['longitude'].values[0]
latitude = ex_site['latitude'].values[0]
altitude = ex_site['altitude'].values[0]
input_ts_fn = examples_filepath+ex_site['input_ts_fn'].values[0]
sim_pars_fn = examples_filepath+ex_site['sim_pars_fn'].values[0].replace('.yml','_benchmark.yml')
H2_demand_fn = examples_filepath+ex_site['H2_demand_col'].values[0]

PPA = 40 # Euro/MWh
hpp = hpp_model(
    latitude,
    longitude,
    altitude,
    num_batteries = 10,
    work_dir = './',
    sim_pars_fn = sim_pars_fn,
    input_ts_fn = input_ts_fn,
    ppa_price = PPA,
    load_min = 3, #MW
    load_min_penalty = 100, #MEuro
    battery_deg = True,
)
x=[35.0, 300.0, 10.0, 3, 7.0, 30, 25.0, 180.0, 1.0, 30, 7, 10.0]


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0


In [11]:
n_procs = int(os.cpu_count())

In [12]:
inputs = {
    'example': 9,
    'name': None,
    'longitude': None,
    'latitude': None,
    'altitude': None,
    'input_ts_fn': None,
    'sim_pars_fn': None,

    'opt_var': "NPV_over_CAPEX",
    'num_batteries': 10,
    'n_procs': n_procs - 1,
    'n_doe': 20,
    'n_clusters': 5,
    'n_seed': 0,
    'max_iter': 2,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 3e4,
    'tol': 1e-6,
    'min_conv_iter': 3,
    'work_dir': './',
    'hpp_model': hpp_model,
    'PPA_price': 40,
    'load_min': 3, # MW
    }
kwargs = get_kwargs(inputs)

Selected example site:
---------------------------------------------------
case                                                 Europe
name                                      Denmark_good_wind
longitude                                          8.594398
latitude                                          56.227322
altitude                                               85.0
input_ts_fn      Europe/GWA3/input_ts_Denmark_good_wind.csv
sim_pars_fn                             Europe/hpp_pars.yml
price_fn                           Europe/2030-EL_PRICE.csv
price_col                                              DK_W
H2_demand_col                          Europe/H2_demand.csv
Name: 9, dtype: object


In [13]:
kwargs['variables'] = {
    'clearance [m]':
        #{'var_type':'design',
        #  'limits':[10, 60],
        #  'types':'int'
        #  },
        {'var_type':'fixed',
           'value': 35
           },
    'sp [W/m2]':
        #{'var_type':'design',
        # 'limits':[200, 359],
        # 'types':'int'
        # },
        {'var_type':'fixed',
           'value': 300
           },
    'p_rated [MW]':
        {'var_type':'design',
          'limits':[1, 10],
          'types':'int'
          },
        # {'var_type':'fixed',
        #  'value': 6
         # },
    'Nwt':
        {'var_type':'design',
          'limits':[0, 400],
          'types':'int'
          },
        # {'var_type':'fixed',
        #   'value': 200
        #   },
    'wind_MW_per_km2 [MW/km2]':
        #{'var_type':'design',
        #  'limits':[5, 9],
        #  'types':'float'
        #  },
         {'var_type':'fixed',
           'value': 7
           },
    'solar_MW [MW]':
        {'var_type':'design',
           'limits':[0, 400],
           'types':'int'
          },
        #{'var_type':'fixed',
        #  'value': 20
        #  },
    'surface_tilt [deg]':
        # {'var_type':'design',
        #   'limits':[0, 50],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 25
          },
    'surface_azimuth [deg]':
        # {'var_type':'design',
        #   'limits':[150, 210],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 180
          },
    'DC_AC_ratio':
        # {'var_type':'design',
        #   'limits':[1, 2.0],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value':1.6,
          },
    'b_P [MW]':
        {'var_type':'design',
           'limits':[0, 100],
           'types':'int'
          },
        #{'var_type':'fixed',
        #  'value': 50
        #  },
    'b_E_h [h]':
        {'var_type':'design',
           'limits':[1, 10],
           'types':'int'
          },
        #{'var_type':'fixed',
        #  'value': 6
        #  },
    'cost_of_battery_P_fluct_in_peak_price_ratio':
        {'var_type':'design',
          'limits':[0, 20],
          'types':'float'
          },
#         {'var_type':'fixed',
#           'value': 10},
    }


In [ ]:
EGOD = EfficientGlobalOptimizationDriver(**kwargs)
EGOD.run()
result = EGOD.result





Sizing a HPP plant at Denmark_good_wind:


Fixed parameters on the site
-------------------------------
longitude = 8.594398
latitude = 56.227322
altitude = 85.0



